In [1]:
import warnings
warnings.simplefilter(action='ignore')


In [3]:
# importation des données

import pandas as pd

data = pd.read_csv("offres_data.csv", index_col=0)

df = pd.DataFrame(data)

# A faire dans l'ordre

1. Filtrer pour garder Data et Donnée(s) (OfferTitle)
2. Supprimer duplicate
3. Ajouter OfferLabel
3. Ajouter Localisations (Région, ville, département)
4. Garder colonnes intéressantes
5. Supprimer duplicate
6. Changer format date
7. Gérer les manquants
8. Refiltrer les offres sur OfferLabel (data, données)
9. Ajout des colonnes ville, code postal etc


# Gestion des valeurs manquantes

- NC = Domaine, ContractType, TeleWork, DisplayedSalary, 
- "" = Profile
- Anonyme = CompanyName
- https://cdn-icons-png.flaticon.com/512/1810/1810755.png = CompanyLogo

# Colonnes à garder 

- Id 
- PublishDate 
- Domaine 
- OfferTitle 
- OfferLabel (Criterions)

- ContractType 
- isFulltime
- Telework
- DisplayedSalary

- Description # Première partie de l'annonce 
- Profile # Seconde Partie de l'annonce 

- CompanyName
- CompanyLogo
- Localisation (a droper pour Localisations)

- UrlOffre

# Besoins

- DisplayedSalary -> Rajouté colonne salaire Min/Max 
- Recherche de hor./semaine
- Localisations -> Pour chopper le max d'infos : Régions, ville, départements sur 3 colonnes ✅
- Recherche NLP des SoftSkills

## 1. Filtrer les annonces Data et Données

In [4]:
import re
df_filtered = df[df['OfferTitle'].str.contains(r"(data|donn[eé]e)", flags=re.IGNORECASE, na=False, regex=True)]


In [5]:
# On passe d'un DF de +21000 à +4800 entrées
df_filtered.shape

(4864, 63)

## 2. Supprimer duplicate

In [6]:
# A refaire plus loin pour vérifier
df_filtered.drop_duplicates(inplace=True)

In [7]:
# On obtient une entrée en moins
df_filtered.shape

(4863, 63)

## 3. Ajouter OfferLabel pour récupérer le type du poste

In [8]:
import ast

def eval(text):
    # liste = []
    ma_liste = ast.literal_eval(text)
    return ma_liste[0]['Label']

df_filtered['OfferLabel'] = df_filtered['Criterions'].apply(eval)

## 4. Garder colonnes intéressantes

In [9]:
df_clean = df_filtered[['Id', 'PublishDate', 'Domaine', 'OfferTitle', 'OfferLabel', 'ContractType', 'IsFulltime', 'Telework', 'DisplayedSalary', 'Description', 'Profile', 'CompanyName', 'CompanyLogo', 'Localisations', 'UrlOffre']]

## 5. Supprimer duplicate (part 2)

In [10]:
df_clean.drop_duplicates(inplace=True)

In [11]:
df_clean.shape

(2141, 15)

## 6. Changer format date

In [12]:
from datetime import datetime
import re

def date_format(date):
    #new_date = datetime.strptime(str(date), "%Y-%m-%d")
    #new_format = date.strftime(date, "%Y-%m-%dT%H:%M:%S")
    regex = re.findall(r"T\d+:\d+:\d+.\d+", date)
    regex = " ".join(regex)
    new_format = date.replace(regex, "")
    return new_format

In [13]:
df_clean['PublishDate'] = df_clean['PublishDate'].apply(date_format)

In [14]:
df_clean['PublishDate'] = df_clean['PublishDate'].apply(lambda x: pd.to_datetime(x))

## 7. Gérer les valeurs manquantes

In [15]:
# On remplace les valeurs vides par NC (non communiqué) sur les colonnes sélectionnées
df_clean['Domaine'].fillna("NC", inplace=True)
df_clean['ContractType'].fillna("NC", inplace=True)
df_clean['Telework'].fillna("NC", inplace=True)
df_clean['DisplayedSalary'].fillna("NC", inplace=True)

In [16]:
# On met une chaine vide à la place des NaN
df_clean['Profile'].fillna("", inplace=True)

In [17]:
# Les entreprises souhaitant rester anonymes
df_clean['CompanyName'].fillna("Anonyme", inplace=True)

In [18]:
# On met un logo par défaut aux annonces qui n'en ont pas
df_clean['CompanyLogo'].fillna("https://cdn-icons-png.flaticon.com/512/1810/1810755.png", inplace=True)

## 8. Refiltrer les offres sur OfferLabel (data, données)

In [19]:
# On refiltre sur OfferLabel pour ne garder que les annonces du type "data", "données"
df_final = df_clean[df_clean['OfferLabel'].str.contains(r"(data|donn[eé]e)", flags=re.IGNORECASE, na=False, regex=True)]


In [20]:
# Résultat
df_final['OfferLabel'].unique()

array(['Ingénieur big data', 'Analyste de données', 'Data manager',
       'Consultant en big data', 'Data miner',
       'Architecte de bases de données', 'Ingénieur bases de données',
       'Architecte big data', 'Data scientist', 'Responsable data LAB',
       'Délégué à la protection des données',
       'Administrateur de bases de données', 'Technicien de données',
       'Ingénieur qualité des données', 'Responsable bases de données',
       'Administrateur data center', 'Technicien data center'],
      dtype=object)

In [21]:
df_final['OfferLabel'].nunique()

17

In [22]:
# On se retrouve avec un jeu de +1600 entrées
df_final.shape

(1693, 15)

## 9. Ajout des colonnes ville, code postal etc

In [23]:
# On crée un df temporaire pour travailler nos données
df_loc = df_final[['Id', 'Localisations']]

In [24]:
# Fonction pour extraire données de localisation

import ast

def extract_coordonnees(text, df_loc):

    def eval_objet():
        ma_liste = []
        if isinstance(text, str):
            ma_liste = ast.literal_eval(text)
            #print(ma_liste)

            pays = None
            region = []
            departement = []
            ville = []
            codepostal = []

            for dico in ma_liste:
                if dico['Type'] == 'Pays':
                    pays = dico['Label']
                if dico['Type'] == 'Region':
                    region.append(dico['Label'])
                if dico['Type'] == 'Departement':
                    departement.append(dico['Label'])
                if dico['Type'] == 'Commune':
                    ville.append(dico['Label'])
                if dico['Type'] == 'Commune':
                    codepostal.append(dico['ShortUri'])
                

            return pays, region, departement, ville, codepostal

        return "NC", "NC", "NC", "NC", "NC"
        
    return eval_objet()


In [25]:
# On ajoute les colonnes souhaitées
df_loc[['Pays', 'Region', 'Departement', 'Ville', 'CodePostal']] = df_loc['Localisations'].apply(lambda x: pd.Series(extract_coordonnees(x, df_loc)))

In [26]:
# On merge ce df temporaire avec le df_final

df_final2 = df_final.merge(df_loc, left_on="Id", right_on="Id",)

In [27]:
# On drop les colonnes indésirables (Localisation_x, Localisation_y)

df_final2.drop(["Localisations_x", "Localisations_y"], axis=1, inplace=True)

In [29]:
# fonction salaire

def salaire(texte):
    dico_s = {}

    if '€ / heure' in texte or '€ / jour' in texte or 'NC' in texte:
        dico_s["salaire_max"] = "NA"
        dico_s["salaire_min"] = "NA"
        
    else:
        if '-' in texte: 
            min_a = texte.split('-')[0].replace('\u202f','').replace(',','.').strip()
            max_a = texte.split('-')[1].replace('\u202f',''""'').replace(',','.').replace(' € / an',"").strip()
            if '€' in max_a:
                min =12*float(min_a)
                max = float(max_a.replace('€ / mois', '').strip())*12
            else :
                min =float(min_a)
                max = float(max_a)

        else : 
            max_a = texte.split('-')[0].replace('\u202f','').replace(',','.').strip().replace(' € / an',"")
            if '€' in max_a:
                max = float(max_a.replace('€ / mois', '').strip())*12
            else :
                max = float(max_a)
            min = max
        dico_s["salaire_max"] = int(max)
        dico_s["salaire_min"] = int(min)

    return dico_s



df_final2['salaire'] = df_final2['DisplayedSalary'].apply(salaire)

df_final2['salaire_min'] = df_final2['salaire'].apply(lambda x : x['salaire_min'])
df_final2['salaire_max'] = df_final2['salaire'].apply(lambda x : x['salaire_max'])


In [31]:
# on supprime la colonne df['salaire'] et on exporte un CSV 
df_final2.drop(['salaire'], axis=1, inplace=True)

df_final2.to_csv('df_final2.csv')